In [45]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

### FUNCTION TO CREATE WIKIPEDIA YEAR URL

In [46]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION: ENSURE ALL MONTH NAMES ARE CONSISTENT (REMOVE SHORTENING)

In [47]:
def correct_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    split_month = x.split(" ")[1]
    #print (split_month)
    
    months_tocheck = ["January", "Febuary ", "March", "April", "May", "June", "July", "August", "September", "October", "Novemeber", "December"]
    
    #if any (split_month for month in months_tocheck):
    if split_month in months_tocheck:
        #print ("it is a correct month")
        return (x)
    
       
    else:
        month_dict = {
        "Mar" : "March",
        "Apr" : "April",
        "May" : "May",
        "June" : "June",
        "July" : "July",
        "Aug" : "August",
        "Sep" : "September",
        "Oct" : "October",
        }
        #print (month_dict[split_month])
        #print ("is not a correct minth")
        time = x[first_space_index:].strip()
        try:
            space_index = time.index(" ")
            hours = time[space_index:].strip()
            return (day + " " + month_dict[split_month] + " " + hours)

        except:
            return (day + " " + month_dict[split_month])


### FUNCTION: RETURN GAMES DATE

In [48]:
def the_date (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    return (day)

### FUNCTION: RETURN GAMES MONTH

In [49]:
def the_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    split_month = x.split(" ")[1]
    return (split_month)


### FUNCTION: RETURN TIME IN HOURS- PERHAPS CONVERT TO 24 HOUR TIME?

In [50]:
def the_time (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (hours[1:-1])
    #print (split_month)

### FUNCTION: RETURN MATCH DAY IF IT EXISTS

In [51]:
def if_dayexists (x):
    ## we know that if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[0].strip()
        return (date)
    else: 
        date = x
        return ("No Day Data Provided")
    


### FUNCTION: TO REMOVE THE DAY FROM THE DATE

In [52]:
def day_conversion (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    

### FUNCTION: TO REMOVE ANY WORDS BEFORE THE DATE (EG. PF1:)

In [53]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### FUNCTION: TO CREATE PANDAS STRIPTIME + DATETIME

In [54]:
def structured_times(x):
    if ':' in x:
        struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
        dt = datetime.fromtimestamp(time.mktime(struct_time))
        return (dt)
    else: 
        struct_time = time.strptime(x, "%Y %d %B")
        dt = datetime.fromtimestamp(time.mktime(struct_time))
        return (dt)

### FUNCTION: MATCH SCORES DICTIONARY (SINGLE LINE DATA)

In [55]:
def scores_dictionary (test):
    
    short_home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
    home_goals= short_home_score.split('.')[0]
    home_behinds= short_home_score.split('.')[1].split('(')[0]
    home_total = (short_home_score.split('.')[1].split('(')[1])[:-1]
    short_away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
    away_goals= short_away_score.split('.')[0]
    away_behinds= short_away_score.split('.')[1].split('(')[0]
    away_total = (short_away_score.split('.')[1].split('(')[1])[:-1]
    
    
    ind_match_dict= {
         "Home_Goals": home_goals,
         "Home_Behinds": home_behinds,
         "Home_Total": home_total,
         "Away_Goals": away_goals,
         "Away_Behinds": away_behinds,
         "Away_Total": away_total}

    return (ind_match_dict)

### FUNCTION: MATCH SCORES DICTIONARY (MULTI LINE DATA- GRAND FINALS)

In [12]:
def scores_dictionary_02 (match_goals):
    

    test_data = match_goals.find_all('td')
    home_score = test_data[1].find('span').string
    home_total =  (home_score.split(" ")[1][1:-1])
    home_goals = (home_score.split(" ")[0]).split(".")[0]
    home_behinds = (home_score.split(" ")[0]).split(".")[1]
    away_score = test_data[3].find('span').string
    away_total =  (away_score.split(" ")[1][1:-1])
    away_goals = (away_score.split(" ")[0]).split(".")[0]
    away_behinds = (away_score.split(" ")[0]).split(".")[1]

    
    ind_match_dict= {
         "Home_Goals": home_goals,
         "Home_Behinds": home_behinds,
         "Home_Total": home_total,
         "Away_Goals": away_goals,
         "Away_Behinds": away_behinds,
         "Away_Total": away_total}

    return (ind_match_dict)

### FUNCTION: TYPE A- GENERAL MATCH INFO  DICTIONARY

In [56]:
def match_dictionary_A(test,this_round, this_year):
    #print (test)
    # FUNCTION TAKE OFF FRONT PREFIX
    time_date= date_conversion(test[0].string)


    ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
    day = if_dayexists (time_date)


    ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
    convert_time = day_conversion(time_date)
    #print (convert_time)

    ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
    correct_time_syn = correct_month (convert_time)



    
    
    ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
    date = the_date(correct_time_syn)
    month = the_month(correct_time_syn)
    if ':' in correct_time_syn:
        time = the_time (correct_time_syn)
    else: 
        time = "No time provided"


    ### ADDS THE CURRENT YEAR
    convert_time_year = (str(this_year)+ " " + convert_time)
    


    ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
    struct_time = structured_times(convert_time_year)

    home_team = test[1].find('a').string
    away_team = test[3].find('a').string
    venue = test[4].find('a').string
    venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
    venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
    final_crowd = venue_crowd_.replace(",", "")
    
    try: 
        match_report_url = test[5].find('a')['href']
        
    except:
        match_report_url = "no match report"
   

    match_dict= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured_Data": struct_time,
     "Home_Team": home_team,
     "Away_Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match_Link": match_report_url}
    
    return (match_dict)

### FUNCTION: TYPE B- GENERAL MATCH INFO  DICTIONARY

In [57]:
def match_dictionary_B(test, this_round, this_year, match_report):


    time_date= date_conversion(test[6].string)

    day = if_dayexists (time_date)

    convert_time = day_conversion(time_date)

    convert_time_year = (str(this_year)+ " " + convert_time)
    
    
    date = convert_time.split(" ")[0]
    month = convert_time.split(" ")[1]
    struct_time = structured_times(convert_time_year)
    home_team = test[0].find('a').string
    
    try:
        home_score = test[1].find('b').string
        
    except:
        ## get rid of a child tag within to get out the string
        home_score = (''.join(text for text in test[1].find_all(text=True,recursive=False) if text.parent.name != "small")).strip()

     
        
    home_goals = (home_score.split(" ")[0]).split(".")[0]
    home_behinds = (home_score.split(" ")[0]).split(".")[1]
    home_total = (home_score.split(" ")[1][1:-1])  
    

    away_team = test[2].find('a').string
    
    try:
        away_score = test[3].find('b').string
        
    except:
        ## get rid of a child tag within to get out the string
        away_score = (''.join(text for text in test[3].find_all(text=True,recursive=False) if text.parent.name != "small")).strip()
        

    away_goals = (away_score.split(" ")[0]).split(".")[0]
    away_behinds = (away_score.split(" ")[0]).split(".")[1]
    away_total = (away_score.split(" ")[1][1:-1])  
    venue = test[4].find('a').string
    final_crowd = test[5].string
    match_report_url = match_report
    time = "No time provided"
    
    


    match_dict_= {
     "Day": day,
     "Date": date,
     "Month": month,
     "Year": this_year,
     "Round": this_round,         
     "Time": time,
     "Structured_Data": struct_time,
     "Home_Team": home_team,
     "Away_Team": away_team,
     "Venue": venue,
     "Crowd": final_crowd,
     "Match_Link": match_report_url,
     "Home_Goals": home_goals,
     "Home_Behinds": home_behinds,
     "Home_Total": home_total,
     "Away_Goals": away_goals,
     "Away_Behinds": away_behinds,
     "Away_Total": away_total}
    
    #print (match_dict)
    return (match_dict_)




### FUNCTION: MATCHDAY FOR LARGE TABLE TYPE

In [58]:
def match_function_A(match, this_round, this_year):


    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        if any (name in this_round for name in namestocheck_good):
            if this_round == "Grand Final" or this_round == "Grand Final Replay":
                match_data = match[0]
            else:
                match_data = match

                 
  
            test = match_data.find_all('td')

            ### check if a score is return 
            test_for_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            if test_for_score:
                standard_dict = match_dictionary_A(test,this_round, this_year)
                scores_dict = scores_dictionary(test)
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)
            
            else: 

                standard_dict = match_dictionary_A(test,this_round, this_year)

                scores_dict = scores_dictionary_02(match[1])
                dictionary = {**standard_dict, **scores_dict}
                return (dictionary)

        
        else:
            print ("no name in scope")

    
    except Exception as e:
        print("this row does not have match data", e)


### FUNCTION: MATCHDAY FOR B- SMALL TABLE

In [59]:
def match_function_B(First_Columm, match,this_year):
    
    test = match.find_all('td')

    
    if First_Columm == 'Home team':

    
    
        if len(test) > 7:
            #print (len(test))
            match_report = test[-1].find('a')['href']
            #print (match_report)
            this_round = "Not Defined"
            test = test [:-1]
            #print (test)
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
        else: 
            #print (len(test))
            #print ("normal")
            match_report = "No Match Report"
            this_round = "Not Defined"
            #print (test)
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
 
    else: 

        if len(test) > 8:
            #match_report = test[10]
            match_report = test[-1].find('a')['href']
            this_round = test[0].string
            test = test [1:-2]
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)
        
        else: 
            match_report = "No Match Report"
            this_round = test[0].string
            test = test [1:]
            dictionary = match_dictionary_B(test, this_round, this_year, match_report)
            #print (dictionary)
            return (dictionary)



    


## Why are we not returning this dictioopnary property?
## maybe we could indent it?


### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE A)

In [60]:
def round_function_A(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    this_round = []
    
 
    if round_head == "Grand Final" or round_head == "Grand Final Replay":
        result= match_function_A(games[2:4],round_head, current_year)
        if result:
            this_round.append(result)
        return (this_round)


    else:  
        round_matchday = games[2:-1]
        for match in round_matchday:
            result= match_function_A(match, round_head, current_year)
            if result:
                this_round.append(result)
                
        return (this_round)

    #print ("this")

### FUNCTION: CREATE DATA  FOR EACH ROUND (LIST TYPE B)

In [61]:
def round_function_B(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')

    this_round = []
    
    
    First_Columm = games[0].find_all('td')[0].find('b').string
          
          
    round_matchday = games[1:]


    
    for match in round_matchday:
        result= match_function_B(First_Columm, match,current_year)
        #print (result)
        if result:
            #print (result)
            this_round.append(result)
    ## only working for finals games- check why they are different
    #print (this_round)
    return (this_round)



## TRY BOTH TABLE TYPES THROUGH EVERY YEAR (2001-2017)

### FUNCTION: LARGE TABLE- A

In [62]:
def table_typeA (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    return (year_data)

### FUNCTION: SMALL TABLE- B

In [63]:
def table_typeB (year):    
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')

    year_data= soup.find("div", {"id": "mw-content-text"}).find_all("table", {"class": "wikitable succession-box"})
    return (year_data)

### FUNCTION: CREATE DATA FOR EACH YEAR + TURN INTO 'MEGALIST'

In [64]:
mega_list = []
## 2007 NOT WORKING 2008 NOT WORKING
# 2002 is a mix
## 22004 only old
test_year = 2002

#for year in list(range(test_year,test_year + 1)):
for year in list(range(2001,2018,1)):

        ## Big Table
        year_data_A = table_typeA (year)
        for current_round in range(len(year_data_A)):
            #print (current_round)
            this_round = round_function_A(current_round, year_data_A, year)
            mega_list.extend(this_round)
            
            
        ## Small Table  
        year_data_B = table_typeB (year)
        for current_round in range(len(year_data_B)):
            this_round = round_function_B(current_round, year_data_B, year)
            mega_list.extend(this_round)
            
    
print (mega_list)


this row does not have match data list index out of range
this row does not have match data list index out of range
no name in scope
this row does not have match data time data '2011 11 March 7:45\xa0pm' does not match format '%Y %d %B (%I:%M %p)'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data list index out of range
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
t

### TURN MEGALIST INTO PANDAS DATAFRAME + REVIEW

In [65]:
df = pd.DataFrame(mega_list)
#df.head(200)
df.shape

(3170, 18)

### BREAK

df.loc[df['Round'] == 'Grand Final']

### IDENTIFY UNIQUE VENUE TYPES

In [66]:
df.Venue.unique()

array(['MCG', 'The Gabba', 'Colonial Stadium', 'Football Park',
       'Subiaco Oval', 'Shell Stadium', 'SCG', 'Optus Oval', 'Gabba',
       'York Park', 'Manuka Oval', 'Skilled Stadium', 'Stadium Australia',
       'AAMI Stadium', 'Telstra Stadium', 'Telstra Dome', 'Marrara Oval',
       'Aurora Stadium', 'Carrara Stadium', 'ANZ Stadium',
       'Etihad Stadium', 'TIO Stadium', 'Gold Coast Stadium', 'Subiaco',
       'Patersons Stadium', 'Metricon Stadium', "Cazaly's Stadium",
       'Adelaide Oval', 'Blundstone Arena', 'Blacktown ISP',
       'Simonds Stadium', 'Škoda Stadium', 'StarTrack Oval Canberra',
       'Westpac Stadium', 'Spotless Stadium', 'StarTrack Oval',
       'TIO Traeger Park', 'Domain Stadium', 'UNSW Canberra Oval',
       'University of Tasmania Stadium', 'Jiangwan Stadium'], dtype=object)

### CREATE CONSISTENT VENUE TYPES (NO DOUBLE UPS)

In [72]:
def rationalise_venue(row):
    di_cities= {"MCG": "MCG",
     "The Gabba": "Gabba",
     "Gabba": "Gabba",
     "Colonial Stadium": "Etihad Stadium",
     "Etihad Stadium": "Etihad Stadium",
     "Telstra Dome": "Etihad Stadium",
     "Optus Oval": "Optus Oval",
     "Adelaide Oval": "Adelaide Oval",
     "Manuka Oval": "Manuka Oval",
     "UNSW Canberra Oval": "Manuka Oval",
     "StarTrack Oval": "Manuka Oval",
     "StarTrack Oval Canberra": "Manuka Oval",
     "Domain Stadium": "Domain Stadium",
     "Subiaco Oval": "Domain Stadium",
     "Subiaco": "Domain Stadium",
     "Patersons Stadium": "Domain Stadium",
     "Metricon Stadium": "Metricon Stadium",
     "Carrara Stadium": "Metricon Stadium",
     "Gold Coast Stadium": "Metricon Stadium",
     "WACA": "WACA",
     "Blundstone Arena": "Blundstone Arena",
     "SCG": "SCG",
     "Spotless Stadium": "Spotless Stadium",
     "Blacktown ISP": "Spotless Stadium",
     "Škoda Stadium": "Spotless Stadium",
     "ANZ Stadium": "ANZ Stadium",
     "Telstra Stadium": "ANZ Stadium",
     "Stadium Australia": "ANZ Stadium",
     "Simonds Stadium": "Simonds Stadium",
     "Skilled Stadium": "Simonds Stadium",
     "Shell Stadium": "Simonds Stadium",
     "York Park": "University of Tasmania Stadium",
     "Aurora Stadium": "University of Tasmania Stadium",
     "University of Tasmania Stadium": "University of Tasmania Stadium",
     "Cazaly's Stadium": "Cazaly's Stadium",
     "TIO Traeger Park": "TIO Traeger Park",
     "TIO Stadium": "TIO Stadium",    
     "Marrara Oval": "TIO Stadium",    
     "Westpac Stadium": "Westpac Stadium",
     "Football Park": "AAMI Stadium",
     "AAMI Stadium": "AAMI Stadium",
     "Jiangwan Stadium": "Jiangwan Stadium",
     "Westpac Stadium": "Westpac Stadium"}
    
    
    return di_cities[row.Venue]

In [73]:
df["Venue"] = df.apply(rationalise_venue, axis=1)
df.Venue.unique()

array(['MCG', 'Gabba', 'Etihad Stadium', 'AAMI Stadium', 'Domain Stadium',
       'Simonds Stadium', 'SCG', 'Optus Oval',
       'University of Tasmania Stadium', 'Manuka Oval', 'ANZ Stadium',
       'TIO Stadium', 'Metricon Stadium', "Cazaly's Stadium",
       'Adelaide Oval', 'Blundstone Arena', 'Spotless Stadium',
       'Westpac Stadium', 'TIO Traeger Park', 'Jiangwan Stadium'], dtype=object)

### CREATE CITIES LIST BASED ON VENUES

In [74]:
def get_city(row):
    di_cities= {"MCG": "Melbourne",
     "Gabba": "Brisbane",
     "Etihad Stadium": "Melbourne",
     "Optus Oval": "Adelaide",
     "Adelaide Oval": "Adelaide",
     "Manuka Oval": "Canberra",
     "WACA": "Perth",
     "Domain Stadium": "Perth",
     "Metricon Stadium": "Gold Coast",
     "Blundstone Arena": "Hobart",
     "SCG": "Sydney",
     "Spotless Stadium": "Western Sydney",
     "ANZ Stadium": "Western Sydney",
     "Simonds Stadium": "Geelong",
     "University of Tasmania Stadium": "Launceston",
     "Cazaly's Stadium": "Cairns",
     "TIO Traeger Park": "Alice Springs",
     "TIO Stadium": "Darwin",    
     "Westpac Stadium": "Wellington",
     "AAMI Stadium": "Adelaide",
     "Jiangwan Stadium": "Shanghai"}
                
    return di_cities[row.Venue]

In [75]:
df["City"] = df.apply(get_city, axis=1)

In [29]:
df.head()

,Away_Behinds,Away_Goals,Away_Team,Away_Total,Crowd,Date,Day,Home_Behinds,Home_Goals,Home_Team,Home_Total,Match_Link,Month,Round,Structured_Data,Time,Venue,Year,City
0,14,13,Melbourne,92,"41,648",8,Wednesday,10,14,Richmond,94,No Match Report,March,Not Defined,2000-03-08,No time provided,MCG,2000,Melbourne
1,14,8,Port Adelaide,62,"43,012",9,Thursday,12,24,Essendon,156,No Match Report,March,Not Defined,2000-03-09,No time provided,Etihad Stadium,2000,Melbourne
2,10,24,West Coast,154,"20,580",10,Friday,15,16,Kangaroos,111,No Match Report,March,Not Defined,2000-03-10,No time provided,MCG,2000,Melbourne
3,17,19,Western Bulldogs,131,"37,222",11,Saturday,18,15,Adelaide,108,No Match Report,March,Not Defined,2000-03-11,No time provided,AAMI Stadium,2000,Adelaide
4,15,19,Geelong,129,"19,777",11,Saturday,11,16,Fremantle,107,No Match Report,March,Not Defined,2000-03-11,No time provided,Domain Stadium,2000,Perth


### CREATE WEATHER STATION ID'S LIST BASED ON CITIES

In [76]:
def get_station_ID(row):
    di_stations= {
    "Melbourne": "1487228352&p_stn_num=086232",
     "Brisbane": "334811501&p_stn_num=040913",
     "Adelaide": "106666407&p_stn_num=023090",
     "Canberra": "986936890&p_stn_num=070246",
     "Perth": "17056912&p_stn_num=009225",         
     "Gold Coast": "323084007&p_stn_num=040190",
     "Hobart": "1768364967&p_stn_num=094030",
     "Sydney": "872874356&p_stn_num=066062",
     "Western Sydney": "872504448&p_stn_num=066048",
     "Geelong": "1519479636&p_stn_num=087162",
     "Launceston": "1664874821&p_stn_num=091237",
     "Cairns": "192373211&p_stn_num=031011",
     "Alice Springs": "48646407&p_stn_num=015590",
     "Darwin": "40154901&p_stn_num=014163",    
     "Wellington": "N/A",
     "Shanghai": "N/A"}
                
    return di_stations[row.City]


### copy city links from below

In [77]:
df["station_ID"] = df.apply(get_station_ID, axis=1)

In [78]:
df.shape

(3170, 20)

In [79]:
def find_url(row):
    Base_Link_01= "http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_nccObsCode=136&p_display_type=dailyDataFile&p_startYear="
    Base_Link_02= "&p_c=-"
    link = Base_Link_01 + str(row.Year) + Base_Link_02 + str(row.station_ID)
    return link
    #print (link)

In [80]:
df["URL"] = df[(df.City != 'Wellington') & (df.City != 'Shanghai')].apply(find_url, axis=1)

In [81]:
df.head()

,Away_Behinds,Away_Goals,Away_Team,Away_Total,Crowd,Date,Day,Home_Behinds,Home_Goals,Home_Team,...,Match_Link,Month,Round,Structured_Data,Time,Venue,Year,City,station_ID,URL
0,13,5,Richmond,43,78253,7,No Day Data Provided,11,17,Essendon,...,no match report,September,Qualifying Finals,2001-09-07 19:45:00,7:45 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
1,8,8,Port Adelaide,56,32380,8,No Day Data Provided,16,12,Brisbane Lions,...,no match report,September,Qualifying Finals,2001-09-08 19:40:00,7:40 pm,Gabba,2001,Brisbane,334811501&p_stn_num=040913,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
2,14,6,Adelaide,50,53982,8,No Day Data Provided,16,17,Carlton,...,no match report,September,Elimination Finals,2001-09-08 14:30:00,2:30 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
3,8,11,Sydney,74,32910,9,No Day Data Provided,15,19,Hawthorn,...,no match report,September,Elimination Finals,2001-09-09 14:30:00,2:30 pm,Etihad Stadium,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...
4,14,7,Carlton,56,83323,15,No Day Data Provided,7,10,Richmond,...,no match report,September,Semi Finals,2001-09-15 14:30:00,2:30 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...


In [84]:
# RETURN THE WEATHER 
def find_rain(row):
    req = requests.get(row.URL)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # FIND ALL TR TAGS IN TBODY BESIDES ONES WITH A TR CLASS NOT EQUAL TO 'GRAPHCELL'
    Relevent_Data = soup.find('tbody').find_all('tr', class_=lambda x: x != 'graphcell')

    # CALL THE RELEVENT DAY TR LIST WITH INDEX NUMBER (EG. 3rd = index 2)
    Day_Rain = Relevent_Data[(int(row.Date) - 1)]
    
    # FIND ALL THE 'TD' OR MONTH TAGS FOR A GIVEN DAY OF THE MONTH
    All_Months_Rain = Day_Rain.find_all('td')
    
    # ASSIGN THE MONTH OF A GIVEN ROW
    Month_Value = (int(row['Structured_Data'].month) - 1)

    # CALL THE RELEVENT MONTH TD LIST WITH INDEX NUMBER (note. we have already created index with necessary index)
    Rain = (All_Months_Rain[Month_Value].string)
    print (row['Structured_Data'], Rain)
    return Rain




In [39]:
df.loc[10]

Away_Behinds                                                       9
Away_Goals                                                        15
Away_Team                                                   St Kilda
Away_Total                                                        99
Crowd                                                         20,476
Date                                                              18
Day                                                         Saturday
Home_Behinds                                                      13
Home_Goals                                                        21
Home_Team                                                    Geelong
Home_Total                                                       139
Match_Link                                           No Match Report
Month                                                          March
Round                                                    Not Defined
Structured_Data                   

In [88]:
df["Rain"] = df[(df.City != 'Wellington') & (df.City != 'Shanghai')].apply(find_rain, axis=1)

2001-09-07 19:45:00 0
2001-09-08 19:40:00 0.2
2001-09-08 14:30:00 0.4
2001-09-09 14:30:00 5.8
2001-09-15 14:30:00 0.2
2001-09-15 19:15:00 0
2001-09-22 14:30:00 0
2001-09-22 19:45:00 0
2001-09-29 14:30:00 0
2001-03-30 00:00:00 0
2001-03-31 00:00:00 0
2001-03-31 00:00:00 0
2001-03-31 00:00:00 0
2001-03-31 00:00:00 0
2001-04-01 00:00:00 0.2
2001-04-01 00:00:00 0
2001-04-01 00:00:00 0
2001-04-06 00:00:00 0
2001-04-07 00:00:00 0
2001-04-07 00:00:00 0
2001-04-07 00:00:00 0
2001-04-07 00:00:00 0
2001-04-08 00:00:00 0.4
2001-04-08 00:00:00 0
2001-04-08 00:00:00 0
2001-04-12 00:00:00 3.4
2001-04-14 00:00:00 0
2001-04-14 00:00:00 0
2001-04-14 00:00:00 0
2001-04-15 00:00:00 0
2001-04-15 00:00:00 0
2001-04-15 00:00:00 0
2001-04-16 00:00:00 0.2
2001-04-20 00:00:00 0
2001-04-20 00:00:00 0.2
2001-04-21 00:00:00 0
2001-04-21 00:00:00 0
2001-04-21 00:00:00 0
2001-04-21 00:00:00 0
2001-04-22 00:00:00 66.4
2001-04-22 00:00:00 28.0
2001-04-25 00:00:00 7.6
2001-04-27 00:00:00 0
2001-04-28 00:00:00 0
2001-0

KeyboardInterrupt: 

df_rain_08 = df[df.Year < 2008]
df_rain.head()
df_rain.shape

df["Rain"] = df_rain_08[(df.City != 'Wellington') & (df.City != 'Shanghai')].apply(find_rain, axis=1)

In [89]:
df.head()

,Away_Behinds,Away_Goals,Away_Team,Away_Total,Crowd,Date,Day,Home_Behinds,Home_Goals,Home_Team,...,Month,Round,Structured_Data,Time,Venue,Year,City,station_ID,URL,Rain
0,13,5,Richmond,43,78253,7,No Day Data Provided,11,17,Essendon,...,September,Qualifying Finals,2001-09-07 19:45:00,7:45 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0
1,8,8,Port Adelaide,56,32380,8,No Day Data Provided,16,12,Brisbane Lions,...,September,Qualifying Finals,2001-09-08 19:40:00,7:40 pm,Gabba,2001,Brisbane,334811501&p_stn_num=040913,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0.2
2,14,6,Adelaide,50,53982,8,No Day Data Provided,16,17,Carlton,...,September,Elimination Finals,2001-09-08 14:30:00,2:30 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0.4
3,8,11,Sydney,74,32910,9,No Day Data Provided,15,19,Hawthorn,...,September,Elimination Finals,2001-09-09 14:30:00,2:30 pm,Etihad Stadium,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,5.8
4,14,7,Carlton,56,83323,15,No Day Data Provided,7,10,Richmond,...,September,Semi Finals,2001-09-15 14:30:00,2:30 pm,MCG,2001,Melbourne,1487228352&p_stn_num=086232,http://www.bom.gov.au/jsp/ncc/cdio/weatherData...,0.2


### SELECT ROW BASED ON COLUMN VALUE (EG.VENUE IS METRICON)

df.loc[df['Venue'] == 'Metricon Stadium']

In [96]:
df.dtypes

Away_Behinds               object
Away_Goals                 object
Away_Team                  object
Away_Total                 object
Crowd                      object
Date                       object
Day                        object
Home_Behinds               object
Home_Goals                 object
Home_Team                  object
Home_Total                 object
Match_Link                 object
Month                      object
Round                      object
Structured_Data    datetime64[ns]
Time                       object
Venue                      object
Year                        int64
City                       object
station_ID                 object
URL                        object
Rain                       object
dtype: object

### EXPORT DATA TO CSV

In [97]:
df.to_csv("afl_wiki_data_set.csv")